# ABECTO Tutorial

ABECTO is an **AB**ox **E**valuation and **C**omparison **T**ool for **O**ntologies. It allows to easily compare and evaluate two or more RDF knowledge bases regarding the contained information. This tutorial provides an introduction to the use of ABECTO.


## Preparation

Before we can start, we need to do a few preparation steps.

ABECTO is running as a HTTP REST service in the background. We will use some provided functions, which hide the raw HTTP requests.

In [ ]:
source("abecto.R")

First, we create some sample files that we will use in the tutorial.

In [ ]:
source1file1 <- writeTempFile("
    BASE         <http://example.org/a/>
    PREFIX :     <http://example.org/a/>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

    :alice rdf:type   :Person ;
           rdfs:label \"Alice\" ;
           :pnr       \"12345\"^^xsd:integer .
")

source1file2 <- writeTempFile("
    BASE         <http://example.org/a/>
    PREFIX :     <http://example.org/a/>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd:  <http://www.w3.org/2001/XMLSchema#>

    :bob rdf:type   :Person ;
         rdfs:label \"Bob\" ;
         :pnr       \"67890\"^^xsd:integer .
")

source2file1 <- writeTempFile("
    BASE         <http://example.org/b/>
    PREFIX :     <http://example.org/b/>
    PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    :alice rdf:type   :Person ;
           rdfs:label \"Alice\" .

    :bob rdf:type   :Person ;
         rdfs:label \"Bob\" .

    :charlie rdf:type   :Person ;
             rdfs:label \"Charlie\" .
")

Now, we start the service. **This might take a few seconds.**

In [ ]:
start()

After the service was started, we are ready to create our ontology evaluation and comparison project.

## Project Setup

First, we create a new ABECTO project and store the returned ID. We can also give the project an arbitrary name.

In [ ]:
project <- createProject("Tutorial Project")

Now, we see our new project in the list of projects.

In [ ]:
listProjects()

We can also request information of a certain project.

In [ ]:
getProject(project)

Furthermore, we can delete projects.

In [ ]:
trashProject <- createProject("Trash Project")
deleteProject(trashProject)
listProjects()

Now, we create a knowledge base object for each knwoledge base we want to include into the project.

In [ ]:
kb1 <- createKnowledgebase(project,"Knowledge Base 1")
kb2 <- createKnowledgebase(project,"Knowledge Base 2")

The knowledge bases appear now in the list of knowledge bases of the project.

In [ ]:
listKnowledgebases(project)

We can also request information of a certain knowledge base.

In [ ]:
getKnowledgebase(kb1)

And we can delete knowledge bases.

In [ ]:
trashKB <- createKnowledgebase(project,"Trash Knowledge Base")
deleteKnowledgebase(trashKB)
listKnowledgebases(project)

In [ ]:
kb1source1  <- createStep("RdfFileSourceProcessor",knowledgebase=kb1)
kb1source2  <- createStep("RdfFileSourceProcessor",knowledgebase=kb1)
kb2source1  <- createStep("RdfFileSourceProcessor",knowledgebase=kb2)

In [ ]:
getStep(kb1source1)

In [ ]:
listSteps(project)

In [ ]:
loadStep(kb1source1,source1file1)
loadStep(kb1source2,source1file2)
loadStep(kb2source1,source2file1)

In [ ]:
categories1 <- createStep("ManualCategoryProcessor",input=list(kb1source1,kb1source2),parameters=list(patterns=list(person="?person <http://www.w3.org/2000/01/rdf-schema#label> ?label .")))
categories2 <- createStep("ManualCategoryProcessor",input=list(kb2source1),parameters=list(patterns=list(person="?person <http://www.w3.org/2000/01/rdf-schema#label> ?label .")))

In [ ]:
mapping <- createStep("JaroWinklerMappingProcessor",input=list(categories1,categories2),parameters=list(threshold=0.9,case_sensitive=FALSE,category="person",variables=list("label")))

In [ ]:
runProject(project,await=TRUE)

In [ ]:
getProcessingResult(getLastProcessing(mapping))